# 機械学習スクラッチ入門
## 【問題1】train_test_split のスクラッチ
まずは、scikit-learnの train_test_split をスクラッチしてみます。以下の雛形をベースに関数を実装してください。

In [ ]:
import random
import numpy as np

In [ ]:
def scratch_train_test_split(X, y, train_size=0.8):
    """検証データを分割する。
    Parameters
    ----------
    X : ndarray
      訓練データ (n_samples, n_features)
    y : ndarray
      正解値 (n_samples,)
    train_size : float
      何割をtrainとするか指定 (0 < train_size < 1)
    Returns
    -------
    X_train : ndarray
      訓練データ (n_samples, n_features)
    X_test : ndarray
      検証データ (n_samples, n_features)
    y_train : ndarray
      訓練データの正解値 (n_samples,)
    y_test : ndarray
      検証データの正解値 (n_samples,)
    """
    
    Xn_sample = X.shape[0] # データのインデックス(行、サンプル)の数を抽出
    Xn_train = int((Xn_sample * train_size)) # そのうち何割を訓練データにするか設定
    Xn_test = Xn_sample - Xn_train # 残りをテストデータに
    
    rng = np.random.RandomState() # インスタンス化（ランダムが欲しい）
    shuffle_n_sample = rng.permutation(Xn_sample) # インデックスをランダムに並び替える
    
    X_test = X[shuffle_n_sample[:Xn_test]] # 並び替えた後先頭からテストデータに
    X_train = X[shuffle_n_sample[Xn_test:]] # 残りを訓練データに
    
    
    yn_sample = y.shape[0] # データのインデックス(行、サンプル)の数を抽出
    yn_train = int((yn_sample * train_size)) # そのうち何割を訓練データにするか設定
    yn_test = yn_sample - yn_train # 残りをテストデータに
    
    shuffle_n_sample = rng.permutation(yn_sample) # インデックスをランダムに並び替える
    
    y_test = y[shuffle_n_sample[:yn_test]] # 並び替えた後先頭からテストデータに
    y_train = y[shuffle_n_sample[yn_test:]] # 残りを訓練データに
    
    return X_train, X_test, y_train, y_test

In [ ]:
test_X = np.arange(25).reshape(5, 5)
test_y = np.arange(100, 125)

In [ ]:
display(scratch_train_test_split(test_X, test_y))

In [ ]:
display(scratch_train_test_split(test_X, test_y))

## 【問題2】 分類問題を解くコードの作成
上記3種類の手法で3種類のデータセットを学習・推定するコードを作成してください。

### 1. iris  

1つ目は事前学習期間同様のirisデータセットです。

2値分類としたいため、以下の2つの目的変数のみ利用します。特徴量は4種類すべて使います。

virgicolorとvirginica

In [ ]:
from sklearn.datasets import load_iris 
iris = load_iris()
iris.data.shape

In [ ]:
import pandas as pd
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.DataFrame(iris.target)
y = y.rename(columns={0 : 'Species'})

In [ ]:
display(X.shape, y.shape)

In [ ]:
X = X[50:]
X.shape

In [ ]:
y1 = y.query('Species == 1') # 先程のコードはDF対応させてない
y2 = y.query('Species == 2')
y = pd.concat([y1, y2], axis=0)
y.shape

In [ ]:
# 正規化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
# データ分別
train_X80, test_X20, train_y80, test_y20 = scratch_train_test_split(X, y.values)

#### ロジスティック回帰(最適化方法はSDG)

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_iris = SGDClassifier(loss='log')
sgd_iris.fit(train_X80, train_y80)

In [ ]:
sgd_iris_prod = sgd_iris.predict(test_X20)
from sklearn.metrics import classification_report
print(classification_report(test_y20, sgd_iris_prod))

全部1って出力してるしめちゃスコア低い  

#### SVM

In [ ]:
from sklearn.svm import SVC
svc_iris = SVC() # probabilityで確率出力
svc_iris.fit(train_X80, train_y80)
svc_iris_pred = svc_iris.predict(test_X20)
print(classification_report(test_y20, svc_iris_pred))

まぁまぁ低い

#### 決定木

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc_iris = DecisionTreeClassifier()
dtc_iris.fit(train_X80, train_y80)
dtc_iris_pred = dtc_iris.predict(test_X20)
print(classification_report(test_y20, dtc_iris_pred))

低い

### 2.シンプルデータセット1

In [ ]:
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X = np.concatenate([f0, f1])
y = np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])

In [ ]:
# データ分別
train_X80, test_X20, train_y80, test_y20 = scratch_train_test_split(X, y)

#### ロジスティック回帰

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_simple1 = SGDClassifier(loss='log')
sgd_simple1.fit(train_X80, train_y80)
sgd_simple1_prod = sgd_simple1.predict(test_X20)
from sklearn.metrics import classification_report
print(classification_report(test_y20, sgd_simple1_prod))

#### SVM

In [ ]:
from sklearn.svm import SVC
svc_simple1 = SVC() # probabilityで確率出力
svc_simple1.fit(train_X80, train_y80)
svc_simple1_pred = svc_simple1.predict(test_X20)
print(classification_report(test_y20, svc_simple1_pred))

#### 決定木

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc_simple1 = DecisionTreeClassifier()
dtc_simple1.fit(train_X80, train_y80)
dtc_simple1_pred = dtc_simple1.predict(test_X20)
print(classification_report(test_y20, dtc_simple1_pred))

### 3.シンプルデータセット2

In [ ]:
X = np.array([
    [-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
    [ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
    [-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
    [ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
    [-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
    [-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
    [-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
    [-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
    [ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
    [-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
    [-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
    [ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
    [ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
    [-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
    [ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
    [-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
    [ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
    [-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
    [ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
    [-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ],
])
y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# データ分別
train_X80, test_X20, train_y80, test_y20 = scratch_train_test_split(X, y)

#### ロジスティック回帰

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_simple2 = SGDCla　ssifier(loss='log')
sgd_simple2.fit(train_X80, train_y80)
sgd_simple2_prod = sgd_simple2.predict(test_X20)
from sklearn.metrics import classification_report
print(classification_report(test_y20, sgd_simple2_prod))

#### SVM

In [ ]:
from sklearn.svm import SVC
svc_simple2 = SVC() # probabilityで確率出力
svc_simple2.fit(train_X80, train_y80)
svc_simple2_pred = svc_simple2.predict(test_X20)
print(classification_report(test_y20, svc_simple2_pred))

#### 決定木

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc_simple2 = DecisionTreeClassifier()
dtc_simple2.fit(train_X80, train_y80)
dtc_simple2_pred = dtc_simple2.predict(test_X20)
print(classification_report(test_y20, dtc_simple2_pred))

## 【問題3】 回帰問題を解くコードの作成
線形回帰でHouse Pricesデータセットを学習・推定するコードを作成してください。

In [ ]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [ ]:
y = train_df['SalePrice']
XX = train_df[['GrLivArea', 'YearBuilt']]

In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(XX)

In [ ]:
# データ分別
train_X80, test_X20, train_y80, test_y20 = scratch_train_test_split(X, y)

#### ロジスティック回帰 minmax

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDClassifier()
sgd_reg.fit(train_X80, train_y80)
sgd_reg_prod = sgd_reg.predict(test_X20)

In [ ]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(test_y20, sgd_reg_prod)) # 平均二乗誤差

In [ ]:
import seaborn as sns
sns.distplot(test_y20)
sns.distplot(sgd_reg_prod)

#### ロジスティック回帰 標準化

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(XX)

In [ ]:
# データ分別
train_X80, test_X20, train_y80, test_y20 = scratch_train_test_split(X, y)

In [ ]:
sgd_reg2 = SGDClassifier()
sgd_reg2.fit(train_X80, train_y80)
sgd_reg2_prod = sgd_reg2.predict(test_X20)

In [ ]:
print(mean_squared_error(test_y20, sgd_reg_prod)) # 平均二乗誤差

In [ ]:
import seaborn as sns
sns.distplot(test_y20)
sns.distplot(sgd_reg2_prod)